In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sales = pd.read_csv('datasets/superstore_sales.csv', parse_dates=['Order Date', 'Ship Date'])

##### EDA
Note the date range
1. Total number of orders
2. Total number of orders created per day; together with total number of orders shipped per day
3. Ship Mode
4. Total number of customers
5. Total number of customers per state
6. Total number of product categories sold
7. Total sales
8. Average sales per product category
9. Average sales per state

In [3]:
max_date = sales['Order Date'].max()

In [4]:
rf_df = sales.groupby('Customer ID').agg(recency = ('Order Date', 'max'),
                                         frequency = ('Order ID', 'nunique'))

In [5]:
rf_df['recency'] = (max_date - rf_df['recency']).dt.days

In [6]:
m_df = sales.groupby(['Customer ID','Order ID']).sum().reset_index()\
    .groupby('Customer ID').agg(monetary = ('Sales', 'median'))

In [7]:
rfm_df = pd.concat([rf_df, m_df], axis=1)

In [8]:
r_rank = rfm_df['recency'].rank(pct=True, ascending=False)
f_rank = rfm_df['frequency'].rank(pct=True)
m_rank = rfm_df['monetary'].rank(pct=True)

In [9]:
ranks = [r_rank, f_rank, m_rank]

In [10]:
bins=[0,0.5,1]
names= ['L','H']
numbers = [0,1]

categories, scores = ({} for _ in range(2))
for rank in ranks:
    category = pd.cut(rank, bins, labels=names)
    score = pd.cut(rank, bins, labels=numbers)
    categories[rank.name] = category.astype(str)
    scores[rank.name] = score.astype(int)

In [11]:
rfm_df['rfm_category'] = categories['recency'] + categories['frequency'] + categories['monetary']

In [12]:
rfm_df['rfm_score'] = scores['recency'] + scores['frequency'] + scores['monetary']

In [13]:
rfm_df.rfm_score.value_counts()

2    286
1    269
3    134
0    104
Name: rfm_score, dtype: int64

In [14]:
rfm_df.rfm_category.value_counts()

HHL    134
HHH    134
LLH    111
LLL    104
LHH     94
LHL     87
HLL     71
HLH     58
Name: rfm_category, dtype: int64

In [15]:
rfm_df.to_csv('datasets/rfm_results.csv')